In [1]:
packages = [
    "torch==2.0.1",
    "torch-geometric==2.3.1",
    "numpy==1.23.0",
    "pandas==1.5.3"
]
import warnings
warnings.filterwarnings("ignore")
# Instalación de cada paquete uno por uno
for package in packages:
    !pip install {package}

  Using cached numpy-1.23.0.tar.gz (10.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build numpy


  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [213 lines of output]
      setup.py:71: RuntimeWarning: NumPy 1.23.0 may not yet support Python 3.11.
        warnings.warn(
      Running from numpy source directory.
      setup.py:86: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils_status_migration.html
      
      
        import numpy.distutils.command.sdist
      Cythonizing sources
      Processing numpy/random\_bounded_integers.pxd.in
      Processing numpy/random\bit_generator.pyx
      Processing numpy/random\mtrand.pyx
      Proces

  Using cached pandas-1.5.3-cp311-cp311-win_amd64.whl (10.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3


In [2]:
!pip install torch
!pip install --upgrade numpy pandas
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch.distributions import Categorical
import numpy as np
import pandas as pd

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3


Input Data Processing

In [6]:
#set input dates
date = "_2024-11-04"
truck = "VAN"

In [7]:
# Specify the file name
file_name1 = "rate_q2_west_"+truck+date+".csv"
file_name2 = "duration_west.csv"
file_name3 = "prob_west_"+truck+date+".csv"
file_name4 = "load_av_west_"+truck+date+".csv"
file_name5 = "distance_west.csv"
file_name6 = "diesel_west"+date+".csv"
file_name7 = "labels_west.csv"
# Read the Excel file into a DataFrame
rate_matrix = pd.read_csv(file_name1,header= None)
time_matrix = pd.read_csv(file_name2,header = None)
markov_matrix = pd.read_csv(file_name3,header=None)
loads_matrix = pd.read_csv(file_name4,header=None)
distance_matrix = pd.read_csv(file_name5,header=None)
diesel_matrix = pd.read_csv(file_name6,header=None)
hub_labels = pd.read_csv(file_name7,header=None)
initial_node = 9
time_limit = 45

In [8]:
revenue_matrix = rate_matrix * distance_matrix
revenue_matrix[loads_matrix <= 1] = 0
diesel_matrix = diesel_matrix / 6
cost_matrix = distance_matrix * diesel_matrix
cost_matrix = cost_matrix + 163
reward_matrix = revenue_matrix - cost_matrix

DRL Graph Environment

In [9]:
class GraphEnvironment:
    def __init__(self, reward_matrix, time_matrix, markov_matrix, initial_node, time_limit,loads_matrix):
        self.reward_matrix = reward_matrix
        self.time_matrix = time_matrix
        self.markov_matrix = markov_matrix
        self.initial_node = initial_node
        self.time_limit = time_limit
        self.loads_matrix = loads_matrix
        self.num_nodes = reward_matrix.shape[0]
        self.edge_index = self.generate_edge_index()

    def generate_edge_index(self):
        edges = []
        for i in range(self.num_nodes):
            for j in range(self.num_nodes):
                if i != j:
                    edges.append((i, j))
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        edge_index = edge_index[1]
        return edge_index

    def reset(self):
        self.current_node = self.initial_node
        self.remaining_time = self.time_limit
        self.visited_nodes = [self.initial_node]
        self.visited_edges = []
        self.total_reward = 0
        return self.get_state()

    def get_state(self):
        return torch.tensor([self.current_node, self.remaining_time], dtype=torch.float32)

    def step(self, action):
        next_node = action.item()
        edge = (self.current_node, next_node)
        if (edge not in self.visited_edges) and (self.current_node != next_node):
            self.visited_edges.append(edge)
            reward = self.reward_matrix.iloc[self.current_node, next_node]
            #loads = self.loads_matrix.iloc[self.current_node, next_node]
            time_consumption = self.time_matrix.iloc[self.current_node, next_node]
            self.total_reward += reward
            self.remaining_time -= time_consumption
            self.current_node = next_node
            self.visited_nodes.append(next_node)
            done = (-self.time_limit*0.5 <= self.remaining_time < self.time_limit) and (self.current_node == self.initial_node)
            if done:
                return self.get_state(), reward, done, {},self.visited_nodes
            else:
              if (len(self.visited_nodes) == self.num_nodes) or (self.remaining_time <= -self.time_limit*0.5):
                self.current_node = self.initial_node
                self.remaining_time = self.time_limit
                self.visited_nodes = [self.initial_node]
                self.visited_edges = []
                self.total_reward = 0
                return self.get_state(), 0, done, {},self.visited_nodes
              elif (-self.time_limit*0.5 < self.remaining_time <= 0):
                edge = (self.current_node, self.initial_node)
                reward = self.reward_matrix.iloc[self.current_node, self.initial_node]
                #loads = self.loads_matrix.iloc[self.current_node, self.initial_node]
                time_consumption = self.time_matrix.iloc[self.current_node, self.initial_node]
                self.total_reward += reward
                self.remaining_time -= time_consumption
                self.current_node = self.initial_node
                self.visited_nodes.append(self.initial_node)
                done = (self.current_node == self.initial_node)
                return self.get_state(), reward, done, {},self.visited_nodes
              else:
                return self.get_state(), reward, done, {},self.visited_nodes
        else:
            return self.get_state(), 0, False, {},self.visited_nodes


Neural Network Decision Making Model

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleDQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleDQN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, output_size)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

Model Training

In [11]:
import torch.optim as optim
import random
env = GraphEnvironment(reward_matrix, time_matrix, markov_matrix, initial_node, time_limit,loads_matrix)

# Define hyperparameters
learning_rate = 0.001
output_size = len(env.edge_index)  # Number of possible actions (edges)
# Initialize QNetwork
# Instantiate the model
input_size = 2  # State dimension (current node, remaining time)
hidden_size = 3192  # You can adjust this size
model = SimpleDQN(input_size, hidden_size, output_size)
# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Define loss function (mean squared error)
loss_fn = nn.MSELoss()
# Define a function to select actions using epsilon-greedy policy
def select_action(state, epsilon):
    if np.random.rand() < epsilon:
        a_tpye=1
        options = env.markov_matrix.iloc[int(state[0].item())]
        probabilities = options / options.sum()
        action = torch.tensor(int(np.random.choice(options.index, p=probabilities.values)+((state[0].item())*(env.num_nodes-1))))
        return action,a_tpye
    else:
        with torch.no_grad():
            q_values = model(state)
            a_tpye=2
            return q_values.argmax(dim=-1).unsqueeze(0) , a_tpye

In [ ]:
for i in range(env.num_nodes):
  stop = 0
  last_reward=0
  episode = 0
  gamma=0.5
  epsilon_start=1.0
  epsilon_decay=0.995
  epsilon_min=0.01
  counter = 100
  best = 1
  epsilon = max(epsilon_start * (epsilon_decay ** episode), epsilon_min)
  while stop < 10 and episode < 750 and epsilon > epsilon_min and counter > 0:
      initial_node=i
      time_limit=45
      env = GraphEnvironment(reward_matrix, time_matrix, markov_matrix, initial_node, time_limit,loads_matrix)
      state = env.reset()
      epsilon = max(epsilon_start * (epsilon_decay ** episode), epsilon_min)
      done = False
      total_reward = 0
      while not done:
          action = select_action(state, epsilon)
          new_node = env.edge_index[action[0].item()]
          next_state, reward, done, _,_  = env.step(new_node)
          total_reward += reward
          if done:
              target = torch.tensor(reward)
          else:
              with torch.no_grad():
                  next_q_values = model(next_state)
                  target = reward + gamma * next_q_values.max()
          target = target.float()
          q_values = model(state)
          loss = F.mse_loss(q_values[action[0].item()], target)
          if loss.dtype != torch.float32:
            loss = loss.float()
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          state = next_state
      if total_reward > best:
          if total_reward > best * 1.1:
              counter = 100  # Reset counter if a substantial improvement is found
          best = total_reward
      else:
          counter -= 1
      if last_reward == total_reward:
        stop = stop+1
      else:
        stop=0
      last_reward = total_reward
      print(f"Episode {episode + 1}, Total Reward: {total_reward}, Node: {i}, Epsilon: {epsilon:.3f}")
      episode = episode + 1

1.0
Episode 1, Total Reward: -1035.1099999999997, Node; 7
0.995
Episode 2, Total Reward: 389.026666666667, Node; 7
0.990025
Episode 3, Total Reward: 1349.3199999999997, Node; 7
0.985074875
Episode 4, Total Reward: -577.2833333333332, Node; 7
0.980149500625
Episode 5, Total Reward: -404.72333333333324, Node; 7
0.975248753121875
Episode 6, Total Reward: 2442.576666666667, Node; 7
0.9703725093562656
Episode 7, Total Reward: 2300.66, Node; 7
0.9655206468094842
Episode 8, Total Reward: -556.3733333333332, Node; 7
0.9606930435754368
Episode 9, Total Reward: 698.4266666666667, Node; 7
0.9558895783575596
Episode 10, Total Reward: 1947.88, Node; 7
0.9511101304657719
Episode 11, Total Reward: 4685.653333333334, Node; 7
0.946354579813443
Episode 12, Total Reward: -1644.1216666666664, Node; 7
0.9416228069143757
Episode 13, Total Reward: 2722.5216666666665, Node; 7
0.9369146928798039
Episode 14, Total Reward: 1488.971666666667, Node; 7
0.9322301194154049
Episode 15, Total Reward: 2349.7183333333332

Display Results and Route Solutions

In [20]:
def calculate_days(route, time_matrix):
  total_time = 0
  for i in range(0,len(route)-1):
    lane_time = time_matrix.iloc[route[i],route[i+1]]
    total_time = total_time + lane_time + 2
  days = int(np.floor(total_time / 11))
  return days

In [21]:
def visualize_route (route,hub_labels):
  visualized_route = []
  for i in range(0,len(route)):
    name = hub_labels.at[route[i], 0]
    visualized_route.append(name)
  return visualized_route

In [22]:
def save_best(route, reward, best_profits, most_profit_routes):
    if reward > best_profits.min():
        # Find the index of the lowest value in best_profits
        min_index = best_profits.idxmin()
        
        # Replace the lowest value with the new reward
        best_profits[min_index] = reward
        
        # Replace the corresponding route in most_profit_routes
        most_profit_routes[min_index] = route
        
    return best_profits, most_profit_routes

In [ ]:
type1=0
type2=0
type3=0
type4=0
travel_times = [25,45,60,75,85,100]
solutions = len(travel_times)
most_profit_routes = [0,0,0,0,0]
best_profits = [1,2,3,4,5]
# Inference loop
for i in [9]:
    for j in range(0,solutions):
      taken_actions = []
      # Reset environment to get initial state
      # Initialize environment
      initial_node=i
      time_limit=travel_times[j]
      env = GraphEnvironment(reward_matrix, time_matrix, markov_matrix, initial_node, time_limit,loads_matrix)
      state = env.reset()
      # Episode loop
      done = False
      q_values = model(state)
      while not done:
          # Select action using the trained model
          with torch.no_grad():
              q_values = model(state)
              # Apply action mask to zero out probabilities of previously taken actions
              action_mask = torch.zeros_like(q_values)
              # Set probabilities of previously taken actions to -infinity
              action_mask[taken_actions] = float('-1000000')
              masked_q_values = q_values + action_mask
              probabilities = F.softmax(masked_q_values, dim=-1)
              action = torch.multinomial(probabilities, num_samples=1).unsqueeze(0)
              #action = masked_q_values.argmax(dim=-1).unsqueeze(0)
          # Take action in the environment
          new_node = env.edge_index[action[0].item()]
          next_state, reward, done, _,route = env.step(new_node)
          # Update current state
          state = next_state
          # Update list of previously taken actions
          taken_actions.append(action[0].item())
          if done :
            total_reward =0
            total_revenue = 0
            feasible = 1
            steps = len(route)-1
            for j in range(0,steps):
              reward = reward_matrix.iloc[route[j],route[j+1]]
              revenue = revenue_matrix.iloc[route[j],route[j+1]]
              total_reward = total_reward + reward
              total_revenue = total_revenue + revenue
              loads = loads_matrix.iloc[route[j],route[j+1]]
              if loads < 2:
                feasible = 0
      if total_reward > 0 and feasible == 1:
        type1=type1+1
        #best_profits, most_profit_routes = save_best(route, total_reward, best_profits, most_profit_routes)
        
      elif total_reward > 0 and feasible == 0:
        type2=type2+1
      elif total_reward <= 0 and feasible == 1:
        type3=type3+1
      else:
        type4=type4+1
      days = calculate_days(route, time_matrix)
      visualized_route = visualize_route(route,hub_labels)
      # Output episode solution
      print("Route Labels:",route,"Profit",total_reward,"Route:",visualized_route,"Feasible:",feasible)

Route Labels: [7, 51, 7] Profit 2293.6000000000004 Route: ['Bakersfield (CA)', 'Fort Worth (TX)', 'Bakersfield (CA)'] Feasible: 1
Route Labels: [7, 51, 46, 7] Profit 407.49000000000046 Route: ['Bakersfield (CA)', 'Fort Worth (TX)', 'Chicago (IL)', 'Bakersfield (CA)'] Feasible: 0
Route Labels: [7, 51, 46, 52, 7] Profit 3691.791666666667 Route: ['Bakersfield (CA)', 'Fort Worth (TX)', 'Chicago (IL)', 'Hazleton (PA, NJ)', 'Bakersfield (CA)'] Feasible: 1
Route Labels: [7, 51, 7] Profit 2293.6000000000004 Route: ['Bakersfield (CA)', 'Fort Worth (TX)', 'Bakersfield (CA)'] Feasible: 1
Route Labels: [7, 51, 7] Profit 2293.6000000000004 Route: ['Bakersfield (CA)', 'Fort Worth (TX)', 'Bakersfield (CA)'] Feasible: 1
Route Labels: [7, 51, 7] Profit 2293.6000000000004 Route: ['Bakersfield (CA)', 'Fort Worth (TX)', 'Bakersfield (CA)'] Feasible: 1
Route Labels: [8, 51, 8] Profit 227.52000000000044 Route: ['Fresno (CA)', 'Fort Worth (TX)', 'Fresno (CA)'] Feasible: 0
Route Labels: [8, 51, 46, 8] Profit 

In [24]:
#Display Confusion matrix
# Create the confusion matrix using a dictionary
data = {
    'Profitable Routes': [type1, type2],
    'Non-Profitable Routes': [type3, type4]
}
# Define the index labels
index_labels = ['Doable Routes', 'Non Doable Routes']
# Create the DataFrame
confusion_matrix = pd.DataFrame(data, index=index_labels)
# Display the DataFrame
print(confusion_matrix)

                   Profitable Routes  Non-Profitable Routes
Doable Routes                     21                      0
Non Doable Routes                  3                      0


In [ ]:
#Comparar Tiempo y Calidad en 3 metricas
#Profit Per Day, % de rutas que salen igual, Tiempo total
#Correr dos regiones iguales en ambos algoritmos y comparar metricas